In [9]:
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
import tensorflow as tf
from time import sleep
from datetime import datetime, date, timedelta, time
from sklearn.preprocessing import MinMaxScaler
import threading
from decimal import *
import tableGetter

df = pd.read_csv('C:\\src\\stockPredictor\\SpyHist.csv', index_col=0)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['date'] = [d.date() for d in df['DateTime']]
df['time'] = [d.time() for d in df['DateTime']]
df['Open'] = df['Open'].astype(float)
df['Close'] = df['Close'].astype(float)
df['Volume'] = df['Volume'].astype(int)
df['Difference'] = df['Close'] - df['Open']
df = df.dropna()

for currentDate in df.date.unique():
    if len(df.date.unique()) > len(df.loc[df['date'] == currentDate]):
        df = df.loc[df['date'] != currentDate]

lastDate = df.date.unique()[-1]
yesterdaysDate = (datetime.today() - timedelta(days=1)).date()
if yesterdaysDate > lastDate:
    class IBapi(EWrapper, EClient):
        def __init__(self):
            EClient.__init__(self, self)
            self.data = [] #Initialize variable to store candle

        def historicalData(self, reqId, bar):
            self.data.append([bar.date, bar.open, bar.close, bar.volume])
            
    def run_loop():
        app.run()

    app = IBapi()
    app.connect('127.0.0.1', 7496, 123)

    #Start the socket in a thread
    api_thread = threading.Thread(target=run_loop, daemon=True)
    api_thread.start()

    sleep(1) #Sleep interval to allow time for connection to server

    #Create contract object
    contract = Contract()
    contract.symbol = 'SPY'
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'

    #Request historical candles
    app.reqHistoricalData(1, contract, yesterdaysDate.strftime('%Y%m%d 16:00:00') + ' US/Eastern', str((yesterdaysDate - lastDate).days) + ' D', '5 mins', 'TRADES', 1, 2, False, [])

    sleep(int((yesterdaysDate - lastDate).days/2)) #sleep to allow enough time for data to be returned

    #change back to app.data
    newWholeData = pd.DataFrame(app.data, columns=['DateTime', 'Open', 'Close', 'Volume'])
    newWholeData['DateTime'] = pd.to_datetime(newWholeData['DateTime'],unit='s').dt.tz_localize(tz='UTC')
    newWholeData['DateTime'] = newWholeData['DateTime'].dt.tz_convert(tz='US/Eastern')
    newWholeData['date'] = [d.date() for d in newWholeData['DateTime']]
    newWholeData['time'] = [d.time() for d in newWholeData['DateTime']]
    newWholeData['Open'] = newWholeData['Open'].astype(float)
    newWholeData['Close'] = newWholeData['Close'].astype(float)
    newWholeData['Volume'] = newWholeData['Volume'].astype(int)
    newWholeData['Difference'] = newWholeData['Close'] - newWholeData['Open']

    df = pd.concat([df, newWholeData[~newWholeData['DateTime'].isin(df['DateTime'])]], ignore_index=True).dropna()

    df.to_csv('SpyHist.csv')  
    app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj
